In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os
import sys
import pprint
pp = pprint.PrettyPrinter(indent=4)
sys.path.append("/home/muncok/DL/projects/sv_system/")

Populating the interactive namespace from numpy and matplotlib


# Dataframes

In [2]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/voxceleb/'
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Voxc_Dataframe.pkl')

In [5]:
## sentence duration ###

import librosa
audio_lengths = []
for idx, row in data_df.sample(n=10000).iterrows():
    file_path = os.path.join(data_dir, row.spk, row.file)
    x, sr = librosa.load(file_path, sr=16000)
    audio_lengths.append(len(x)/16000)

In [6]:
mean(audio_lengths)

8.2367185

### Dataset split

In [7]:
#TODO: correct splits
all_spks = data_df.spk.unique()
uttrs_counts = data_df.spk.value_counts()

In [9]:
# # for normal si train
# import random
# si_spks = list(uttrs_counts.index[:-30])
# sv_spks = list(uttrs_counts.index[-30:])
# si_df = data_df[data_df.spk.isin(si_spks)]
# si_train = si_df.sample(frac=0.8)
# si_val = si_df.drop(index=si_train.index)
# si_test = si_val
# sv_df = data_df[data_df.spk.isin(sv_spks)]
# print("[si] train:{}, val:{}, test:{}".format(len(si_train), len(si_val), len(si_test)))
# print("[sv]: {}".format(len(sv_df)))

In [9]:
si_train = data_df[data_df.set == 'train']
si_val = data_df[data_df.set == 'val']
si_test = data_df[data_df.set == 'test']
print("[si] train:{}, val:{}, test:{}".format(len(si_train), len(si_val), len(si_test)))

[si] train:137553, val:6816, test:8186


In [11]:
len(all_spks)

1251

In [10]:
len(si_train) + len(si_val) + len (si_test)

152555

In [12]:
si_train.to_pickle("../manifests/voxc/official_split/si_train.pkl")
si_val.to_pickle("../manifests/voxc/official_split/si_val.pkl")
si_test.to_pickle("../manifests/voxc/official_split/si_test.pkl")
# sv_df.to_pickle("manifests/voxc/sv_df.pkl")

In [38]:
# # for fewshot
# import random
# # valid_spks = uttrs_counts[uttrs_counts > 20].index
# # random_idx = np.random.permutation(np.arange(len(valid_spks)))
# # fewshot_train_spks = valid_spks[random_idx[:1200]]
# # fewshot_val_spks = list(valid_spks[1200:])
# fewshot_train = data_df[data_df.spk.isin(fewshot_train_spks)]
# fewshot_val = data_df[data_df.spk.isin(fewshot_val_spks)]
# print("[fewshot] train:{}, val:{}".format(len(fewshot_train), len(fewshot_val)))

[fewshot] train:146605, val:2365


### Write Manifests

In [13]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_train, 'val':si_val, 'test':si_test}
manifest_dir = "../manifests/voxc/official_split/"
all_spks = list(all_spks)
for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("voxc", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = all_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

../manifests/voxc/official_split/si_voxc_train_manifest.csv was written
../manifests/voxc/official_split/si_voxc_val_manifest.csv was written
../manifests/voxc/official_split/si_voxc_test_manifest.csv was written


In [13]:
import csv
import random

tags = ['sv']
# choose a si split
sets = {'sv':sv_df}
manifest_dir = "manifests/voxc/"
all_spks = list(all_spks)
for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'sv_{}_manifest.csv'.format("voxc"))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = all_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

manifests/voxc/sv_voxc_manifest.csv was written
